<center>
<br />
<h1>Code optimization, Numpy, cython</h1>
<h3>Python</h3>
<br />
<h4>2021</h4> </center>

# Matrix

Create Matrix class:

In [1]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (len(self), len(self[0])))

The simplest implementation of matrix multiplication:

In [2]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

 

Let's check its correctness:

In [3]:
X = Matrix([[1], [2], [3]])
Y = Matrix([[4, 5, 6]])
print(matrix_product(X, Y))
print(matrix_product(Y, X))


[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
[[32]]


How much time does it take?

In [4]:
def setup(shape=(100, 100)):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    return X, Y

In [5]:
%%timeit
matrix_product(*setup())

178 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Woah! Almost an entire second! That's too long. <br>
Let's find out where we waste time, for that we can use line_profiler:

In [ ]:
# !pip3 install line_profiler

# If failed:
# git clone https://github.com/rkern/line_profiler.git
# find line_profiler -name '*.pyx' -exec cython {} \;
# cd line_profiler && pip3 install . --user 

In [6]:
%load_ext line_profiler

In [7]:
%lprun -f matrix_product matrix_product(*setup())

```
Timer unit: 1e-06 s

Total time: 2.33726 s
File: <ipython-input-4-b9cb39bb9b1b>
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product(X, Y):
     2         1         13.0     13.0      0.0      n_xrows, n_xcols = X.shape
     3         1          4.0      4.0      0.0      n_yrows, n_ycols = Y.shape
     4                                               
     5         1          2.0      2.0      0.0      assert n_xcols == n_yrows, "Incompatible matrix dimensions"
     6                                               
     7         1        218.0    218.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     8                                               
     9       101         76.0      0.8      0.0      for i in range(n_xrows):
    10     10100       7109.0      0.7      0.3          for j in range(n_xcols):
    11   1010000     740164.0      0.7     31.7              for k in range(n_ycols):
    12   1000000    1589675.0      1.6     68.0                  Z[i][k] += X[i][j] * Y[j][k]
    13         1          1.0      1.0      0.0      return Z
```

We see that a lot of time is spent on for-loop and getting indexes. What do you think can be improved?

In [8]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [9]:
%%timeit
matrix_product(*setup())

181 ms ± 4.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Let's try to remove some element retrieval by index:

In [10]:
def matrix_product2(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Z[i][k] = acc
    return Z

In [11]:
%%timeit
matrix_product2(*setup())

99.2 ms ± 657 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


2 times faster! Since it became about 2 times less operations of retrieval by index.

In [12]:
%lprun -f matrix_product2 matrix_product2(*setup())

```
Timer unit: 1e-06 s

Total time: 2.01848 s
File: <ipython-input-12-e24c1e2d67bf>
Function: matrix_product2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product2(X, Y):
     2         1         11.0     11.0      0.0      n_xrows, n_xcols = X.shape
     3         1          3.0      3.0      0.0      n_yrows, n_ycols = Y.shape
     4         1        109.0    109.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5       101         73.0      0.7      0.0      for i in range(n_xrows):
     6       100        102.0      1.0      0.0          Xi = X[i]
     7     10100       7580.0      0.8      0.4          for k in range(n_ycols):
     8     10000       7562.0      0.8      0.4              acc = 0
     9   1010000     738306.0      0.7     36.6              for j in range(n_xcols):
    10   1000000    1254721.0      1.3     62.2                  acc += Xi[j] * Y[j][k]
    11     10000      10016.0      1.0      0.5              Z[i][k] = acc
    12         1          1.0      1.0      0.0      return Z
```

Let's go further and try to be even faster!

In [13]:
def matrix_product3(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # :)
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            getX = Xi.__getitem__
            Zi[k] = sum(getX(j) * Ytk[j] for j in range(n_xcols))
    return Z

In [14]:
X = [[1],[2],[3]]
Y = [[4,5,6]]
Z = [[0,0,0],[0,0,0],[0,0,0]]
Yt = [[4],[5],[6]]

for i, (Xi, Zi) in enumerate(zip(X, Z)):
        print((i,Xi,Zi))
        for k, Ytk in enumerate(Yt):
            print((k,Ytk))

(0, [1], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])
(1, [2], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])
(2, [3], [0, 0, 0])
(0, [4])
(1, [5])
(2, [6])


In [15]:
%%timeit
matrix_product3(*setup())

79.1 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Again almost 2 times faster!

Conclusion: if your code is slow, do not rush to use other frameworks. Often you can speed up your code at times with little manipulation.

# Numpy

Let's try to take our first implementation, but use an array from numpy instead (we'll see how it works later)

In [16]:
import numpy as np

def np_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [17]:
def setup_np(shape=(100, 100)):
    X = np.random.randint(-255, 255, size=shape)
    Y = np.random.randint(-255, 255, size=shape)
    return X, Y

In [18]:
%timeit np_matrix_product(*setup_np())

300 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


No profit so far.

In [19]:
X, Y = setup_np()

In [20]:
%timeit np.dot(X, Y)

361 µs ± 6.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


It's getting interesting. Good motivation to learn the numpy module.

## Array

In [21]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

print(a.size, a.shape, a.dtype)
print(a)

4 (4,) float64
[4. 5. 2. 3.]


You can see that the array is effectively stored in memory:

In [22]:
print(a.nbytes)
a = np.array([4, 1.1, 2])
print(a.nbytes)

32
24


Shape can be changed:

In [23]:
a = np.array([4, 1.1, 2, 3])

a[1] = 5

a.shape = (2, 2)
a

array([[4., 5.],
       [2., 3.]])

An array is of one type. If you create differently typed elements, they all will be cast to the same type.

In [24]:
np.array([4, 1.1, 2, 3, "string"])

array(['4', '1.1', '2', '3', 'string'], dtype='<U32')

## UFuncs

UFuncs (universal functions) are functions that work element by element.

In [25]:
a = range(4)
b = [value + 1 for value in a]
print(b)

[1, 2, 3, 4]


In numpy:

In [27]:
a = np.arange(4)
print(type(a))
b = a + 1
print(b)

<class 'numpy.ndarray'>
[1 2 3 4]


The addition of two arrays.

[SMTH ABOUT TIMEIT](https://docs.python.org/3/library/timeit.html)

[smth about jupyter magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

In [29]:
%%timeit

a = [random.random() for _ in range(int(1e6))]
b = [random.random() for _ in range(int(1e6))]

127 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit a = [random.random() for _ in range(int(1e6))]
%timeit b = [random.random() for _ in range(int(1e6))]

63.9 ms ± 688 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
65.3 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
count = int(1e7)
a, b = [random.random() for _ in range(count)], [random.random() for _ in range(count)]

%timeit [i + j for (i, j) in zip(a, b)]

370 ms ± 7.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
a, b = np.array(a), np.array(b)
%timeit a + b

9.36 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


It's because numpy is written in c.

Other UFuncs

In [36]:
def slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit slow()

1.8 ms ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
def fast():
    a = np.arange(10000)
    return a ** 2

%timeit fast()

6.69 µs ± 19.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


There are many other UFuncs: arithmetic operations, bitwise, comparisons, trigonometric, etc.

## Aggregation

Numpy has many built-in aggregation functions

<img src="aggregations.png" width=600px>

In [39]:
print(any([True, False, False, False, False, False]))
print(any([False, False, False, False, False, False]))
print(all([True, True, True, True, True]))
print(all([False, True, True, True, True]))

True
False
True
False


In [40]:
data = [random.random() for _ in range(1000000)]
%timeit min(data)

9.52 ms ± 98 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
np_data = np.array(data)
%timeit np.min(np_data)

1.91 ms ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Indexing

There are all basic slices and indexes. But there are many additional tasty features:

In [43]:
np.arange(6)

array([0, 1, 2, 3, 4, 5])

In [47]:
X = np.arange(6).reshape((2, 3))
X

array([[0, 1, 2],
       [3, 4, 5]])

In [45]:
# just an example
# X = np.arange(6).reshape((2, 1, 3))
# X

array([[[0, 1, 2]],

       [[3, 4, 5]]])

In [48]:
X[1]

array([3, 4, 5])

In [50]:
a = [[j for j in range(5)] for _ in range(5)]
a

[[0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4]]

In [53]:
print(f'a[1][1] = {a[1][1]}')
a[1, 1]

a[1][1] = 1


TypeError: list indices must be integers or slices, not tuple

In [49]:
X[0, 1]

1

In [55]:
X

array([[0, 1, 2],
       [3, 4, 5]])

In [54]:
z = X[:1, 1:]
z

array([[1, 2]])

Unlike with standard lists, numpy slices are not copies and you can change the initial list.

In [56]:
a[:]

[[0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4]]

In [57]:
z[:1, :1] = 100

In [58]:
X

array([[  0, 100,   2],
       [  3,   4,   5]])

### Masking

In [59]:
a = np.arange(8) ** 2
a[a > 8]

array([ 9, 16, 25, 36, 49])

This works because numpy.array supports boolean mask indexing:

In [61]:
print(a)
print(a > 8)
print(a[a > 8])

[ 0  1  4  9 16 25 36 49]
[False False False  True  True  True  True  True]
[ 9 16 25 36 49]


You can also use an array of indices:

In [62]:
a = np.arange(4) * 2
print(a)
a[[3, 3, 2, 2, 0, 0, 1, 1]]

[0 2 4 6]


array([6, 6, 4, 4, 0, 0, 2, 2])

## Broadcasting
Or automatic casting of dimensions.

You can stack arrays of different sizes where it makes sense:

In [64]:
a = np.arange(3)
a

array([0, 1, 2])

In [65]:
b = np.array(((0, 0, 0), (10, 10, 10), (20, 20, 20), (30, 30, 30)))
b

array([[ 0,  0,  0],
       [10, 10, 10],
       [20, 20, 20],
       [30, 30, 30]])

In [70]:
a1, b1 = np.array([a, a, a, a]), b.copy()
a1, b1

(array([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]]),
 array([[ 0,  0,  0],
        [10, 10, 10],
        [20, 20, 20],
        [30, 30, 30]]))

In [66]:
b + a

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

In [75]:
b3, a3 = b[:, 0].reshape(-1, 1), a.copy()
b3, a3

(array([[ 0],
        [10],
        [20],
        [30]]),
 array([0, 1, 2]))

In [76]:
b3 + a3

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

In [77]:
b4, a4 = b[:, 0].reshape(1, -1), a.copy()
b4, a4

(array([[ 0, 10, 20, 30]]), array([0, 1, 2]))

In [78]:
b4 + a4

ValueError: operands could not be broadcast together with shapes (1,4) (3,) 

<img src="https://i.stack.imgur.com/JcKv1.png" width=900px/>

But it does not always work as we would like:

In [79]:
X = np.arange(6).reshape((3, 2))
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [80]:
X + np.array([1, 2, 3])

ValueError: operands could not be broadcast together with shapes (3,2) (3,) 

Of course, we understand what was meant, but numpy did not understand. We will tell him:

In [81]:
Y = np.array((1, 2, 3))
print(Y.shape)
print(Y)
Y = np.array((1, 2, 3))[:, np.newaxis]  # reorient array
print(Y.shape)
print(Y)

(3,)
[1 2 3]
(3, 1)
[[1]
 [2]
 [3]]


In [84]:
print(f'X = {X}, \nY = {Y}')
X + Y

X = [[0 1]
 [2 3]
 [4 5]], 
Y = [[1]
 [2]
 [3]]


array([[1, 2],
       [4, 5],
       [7, 8]])

Conclusion: Loops in python are slow. It is worth using the built-in numpy functions soo the code is more understandable and significant acceleration is gained.

# Cython 
Yet another way to optimize your code

The Cython compiler translates a program written in Python or Cython into more efficient C code

Cython is a Python add-on, supporting C function calls and assignment C language data type variables

Let's see how it works. Let's create a couple of functions in pure python:

In [85]:
def f(x):
    return x**2

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [86]:
%%timeit
integrate_f(0, 10, 1000000)

117 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Enabling Cython magic:

In [87]:
%load_ext Cython

In [88]:
%%cython -a 
def f(x):
    return x**2

def integrate_f2(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

Cython is trying to determine variable types and due to this speed up. But those lines of code where he could not establish the type are highlighted in yellow. Almost everything is yellow.

In [89]:
%%timeit
integrate_f2(0, 10, 1000000)

79.5 ms ± 2.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


It's a bit faster, but we can do better.

Let's apply static typing:

In [128]:
%%cython -a 
def f(double x):
    return x**2

def integrate_f3(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # when i is defined as int the loop is compiled into pure C code
        s += f(a + i * dx)
    return s * dx

In [129]:
%%timeit
integrate_f3(0, 10, 1000000)

25.8 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Function calls in python can be expensive. For example, in the last example, there was a recurrent conversion from C double to Python float. 

Let's change the prototype of the function f:

In [154]:
%%cython -a 
cdef double f2(double x):
    return x**2

def integrate_f4(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b - a) / N
    for i in range(N):  # when i is defined as int the loop is compiled into pure C code
        s += f2(a + i * dx)
    return s * dx

In [155]:
%%timeit
integrate_f4(0, 10, 1000000)

1.02 ms ± 126 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Let's compare with numpy

In [100]:
np.linspace(10, 20, 11)

array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.])

In [156]:
%%timeit
x = np.linspace(0, 10, 1_000_000)
(x**2).sum() / 100000

1.1 ms ± 46.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


With Cython you can achieve more than with numpy!

## Matrix
Back to our Matrix example

In [102]:
%%cython -a 
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        n_rows, n_cols = shape
        return cls([[0] * n_cols for i in range(n_rows)])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M

    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return ((0, 0) if not self else
                (int(len(self)), int(len(self[0]))))


def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()
    for i, Xi in enumerate(X):
        for k, Ytk in enumerate(Yt):
            Z[i][k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z


/Users/isklonin/.ipython/cython/_cython_magic_410c865b3ee4bdfff2a0fe7e1060b642.c:3324:3: warning: 'tp_print' is deprecated [-Wdeprecated-declarations]
  0, /*tp_print*/
  ^
/opt/homebrew/Caskroom/miniforge/base/envs/env3.8/include/python3.8/cpython/object.h:260:5: note: 'tp_print' has been explicitly marked deprecated here
    Py_DEPRECATED(3.8) int (*tp_print)(PyObject *, FILE *, int);
    ^
/opt/homebrew/Caskroom/miniforge/base/envs/env3.8/include/python3.8/pyport.h:515:54: note: expanded from macro 'Py_DEPRECATED'
#define Py_DEPRECATED(VERSION_UNUSED) __attribute__((__deprecated__))
                                                     ^
/Users/isklonin/.ipython/cython/_cython_magic_410c865b3ee4bdfff2a0fe7e1060b642.c:3438:3: warning: 'tp_print' is deprecated [-Wdeprecated-declarations]
  0, /*tp_print*/
  ^
/opt/homebrew/Caskroom/miniforge/base/envs/env3.8/include/python3.8/cpython/object.h:260:5: note: 'tp_print' has been explicitly marked deprecated here
    Py_DEPRECATED(3.8) int 

The whole thing is yellow!

In [103]:
shape = (100, 100)

In [104]:
X = Matrix.random(shape)
Y = Matrix.random(shape)

In [105]:
%timeit cy_matrix_product(X, Y)

45.6 ms ± 153 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


It is faster than our Python implementation, but so far it does not reach the numpy version. <br>
Let's try to tell cython what types we use, maybe it will be faster.

In [106]:
X = np.random.randint(-255, 255, size=shape, dtype=np.int64)
Y = np.random.randint(-255, 255, size=shape, dtype=np.int64)

In [107]:
%%cython -a
import numpy as np

def cy_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In [108]:
%timeit cy_matrix_product(X, Y)

284 ms ± 2.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now it's worse!

Cython can't work with Python imports and now it's required to ask python for every little snippet of code.

Let's explain data types for Cython explicitly.

In [109]:
%%cython -a
import numpy as np
cimport numpy as np

def cy_matrix_product2(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In file included from /Users/isklonin/.ipython/cython/_cython_magic_b524a700032082c9adf92784aaa0455d.c:644:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.


In [111]:
%timeit  cy_matrix_product2(X, Y)

289 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It seems that cython now understands more, but it's still slow, since the main parts are still not clear.

Let's add static typing for input values - now our function takes only values of a specified type.

In [112]:
%%cython -a
import numpy as np
cimport numpy as np  # Not every module may be imported like this


def cy_matrix_product3(np.ndarray[np.int64_t, ndim=2] X,
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In file included from /Users/isklonin/.ipython/cython/_cython_magic_9e74d256f1f86b96d6e506e78edc89e4.c:645:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.


In [113]:
%timeit cy_matrix_product3(X, Y)

2.71 ms ± 12 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Well, now we’ll add quite a bit of black magic. <br>
Add decorators that check the boundaries of arrays and type overflows.

In [114]:
%%cython -a
import numpy as np

cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.overflowcheck(False)
def cy_matrix_product4(np.ndarray[np.int64_t, ndim=2] X, 
                      np.ndarray[np.int64_t, ndim=2] Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z = \
        np.zeros((n_xrows, n_ycols), dtype=np.int64)
    for i in range(n_xrows):        
        for k in range(n_ycols):
            for j in range(n_xcols):
                Z[i, k] += X[i, j] * Y[j, k]
    return Z

In file included from /Users/isklonin/.ipython/cython/_cython_magic_99db15a8d2aad6f8bf041cc69cb9b89b.c:646:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/opt/homebrew/Caskroom/miniforge/base/envs/env3.8/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.


In [115]:
%timeit cy_matrix_product4(X, Y)

360 µs ± 3.31 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Profit.

# Memory Usage (just in case)

In [ ]:
#!pip install memory_profiler

In [116]:
%load_ext memory_profiler

ModuleNotFoundError: No module named 'memory_profiler'

## 1) memit

In [ ]:
%%memit
a = [0] * 2*10**8
del a

## 2.1) jupyter mprun

In [ ]:
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

%mprun -T mprof0 -f my_func my_func()

In [ ]:
%%writefile memscript.py
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

In [ ]:
from memscript import my_func
%mprun -T mprof0 -f my_func my_func()

In [ ]:
!cat mprof0

## 2.2) Terminal mprun

In [ ]:
%%writefile memscript2.py
@profile
def my_func():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

my_func()

In [ ]:
!cat memscript2.py

In [ ]:
!python -m memory_profiler memscript2